In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

In [2]:
NUM_CLINICS = 250
CLINIC_CAPACITY = 1000
ALPHA = 0.05
BETA = 0.02
ITERATIONS = 50
LCS_WEIGHT = 0.1

In [3]:
FEATURE_WEIGHTS = {
    'density_area': 0.1376310863,
    'clinick_distance': 0.2639910152,
    'client_age': 0.1343461970,
    'park_distance': 0.1339970075,
    'vulnerable_group_density': 0.1639669107,
    'social_infrastructure_rating': 0.1660677833
}

In [13]:
df = pd.read_csv('clinic-data.csv')
client_coords = df[['x', 'y']].values
FEATURE_COLS = [
    'density_area','clinick_distance','client_age','park_distance',
    'vulnerable_group_density','social_infrastructure_rating'
]
features = df[FEATURE_COLS].values
M = len(df)

In [5]:
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

In [6]:
feature_means = features_scaled.mean(axis=0)
weights = feature_means / feature_means.sum()
lcs_scores = features_scaled @ weights

In [7]:
print("Automatically selected feature weights to maximize LCS:")
for col, w in zip(FEATURE_COLS, weights):
    print(f"{col}: {w:.4f}")

Automatically selected feature weights to maximize LCS:
density_area: 0.1670
clinick_distance: 0.1662
client_age: 0.1663
park_distance: 0.1663
vulnerable_group_density: 0.1673
social_infrastructure_rating: 0.1670


In [8]:
dists = cdist(client_coords, client_coords)
nearest_dists = np.min(dists, axis=1)
T_initial = np.mean(ALPHA * nearest_dists + BETA * nearest_dists**2)
print(f"Initial objective function (average travel cost, middle path): {T_initial:.4f}")

Initial objective function (average travel cost, middle path): 0.0000


In [9]:
kmeans = KMeans(n_clusters=NUM_CLINICS, random_state=42)
kmeans.fit(client_coords, sample_weight=1 + LCS_WEIGHT * lcs_scores)
clinic_coords = kmeans.cluster_centers_
clinic_coords[:, 0] = np.clip(clinic_coords[:, 0], -1000, 1000)
clinic_coords[:, 1] = np.clip(clinic_coords[:, 1], -1000, 1000)

D:\jupyter notebook\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [10]:
for iteration in range(ITERATIONS):
    print(f"Iteration {iteration+1}/{ITERATIONS}")

    clinic_assignments = {i: [] for i in range(NUM_CLINICS)}
    for i, client in enumerate(client_coords):
        dists_to_clinics = np.linalg.norm(clinic_coords - client, axis=1)
        sorted_clinics = np.argsort(dists_to_clinics)
        for c_idx in sorted_clinics:
            if len(clinic_assignments[c_idx]) < CLINIC_CAPACITY:
                clinic_assignments[c_idx].append(i)
                break

    new_clinics = []
    for c_idx, client_indices in clinic_assignments.items():
        if len(client_indices) == 0:
            new_clinics.append(clinic_coords[c_idx])
            continue
        assigned_clients = client_coords[client_indices]
        lcs_assigned = lcs_scores[client_indices]
        weights_centroid = 1 + LCS_WEIGHT * lcs_assigned
        centroid = np.average(assigned_clients, axis=0, weights=weights_centroid)
        centroid[0] = np.clip(centroid[0], -1000, 1000)
        centroid[1] = np.clip(centroid[1], -1000, 1000)
        new_clinics.append(centroid)
    clinic_coords = np.array(new_clinics)

Iteration 1/50
Iteration 2/50
Iteration 3/50
Iteration 4/50
Iteration 5/50
Iteration 6/50
Iteration 7/50
Iteration 8/50
Iteration 9/50
Iteration 10/50
Iteration 11/50
Iteration 12/50
Iteration 13/50
Iteration 14/50
Iteration 15/50
Iteration 16/50
Iteration 17/50
Iteration 18/50
Iteration 19/50
Iteration 20/50
Iteration 21/50
Iteration 22/50
Iteration 23/50
Iteration 24/50
Iteration 25/50
Iteration 26/50
Iteration 27/50
Iteration 28/50
Iteration 29/50
Iteration 30/50
Iteration 31/50
Iteration 32/50
Iteration 33/50
Iteration 34/50
Iteration 35/50
Iteration 36/50
Iteration 37/50
Iteration 38/50
Iteration 39/50
Iteration 40/50
Iteration 41/50
Iteration 42/50
Iteration 43/50
Iteration 44/50
Iteration 45/50
Iteration 46/50
Iteration 47/50
Iteration 48/50
Iteration 49/50
Iteration 50/50


In [18]:
def compute_metrics(clients, clinics, assignments, lcs_scores):
    ttt = 0
    co = 0
    for c_idx, client_indices in assignments.items():
        assigned_clients = clients[client_indices]
        dists = np.linalg.norm(assigned_clients - clinics[c_idx], axis=1)
        ttt += np.sum(ALPHA * dists + BETA * dists**2)
        co += max(0, len(client_indices) - CLINIC_CAPACITY)
        t = ttt / M + co
    lcs = np.mean(lcs_scores[:NUM_CLINICS])
    return ttt, co, lcs, t

ttt, co, lcs, t = compute_metrics(client_coords, clinic_coords, clinic_assignments, lcs_scores)
print(f"TTT: {ttt:.2f}")
print(f"CO: {co}")
print(f"LCS: {lcs:.4f}")
print(f"T: {t:.4f}")

TTT: 2355.84
CO: 0
LCS: 0.5007
T: 0.0561
